<h1>タイタニック @kaggle勉強会 11/11</h1>
<h2>手法：UnderSampling + Bagging + NN</h2>


In [108]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader , TensorDataset

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE 

import pandas as pd
import numpy as np

In [109]:
data = pd.read_csv("./data/train.csv").drop('Ticket',axis=1).drop('Cabin',axis=1).drop('Name',axis=1)
data["Sex"][data["Sex"] == "male"] = 0
data["Sex"][data["Sex"] == "female"] = 1
print(data.shape)
data.head()

(891, 9)


/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,S
1,2,1,1,1,38.0,1,0,71.2833,C
2,3,1,3,1,26.0,0,0,7.9250,S
3,4,1,1,1,35.0,1,0,53.1000,S
4,5,0,3,0,35.0,0,0,8.0500,S


In [110]:
data["Age"] = data["Age"].fillna(data["Age"].median())
data["Embarked"] = data["Embarked"].fillna("S")


data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [111]:
dummy_columns = ['Embarked']

dummy = pd.get_dummies(data[dummy_columns])
data = pd.concat([data, dummy], axis=1).drop('Embarked',axis=1).astype(float)

data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1.0,0.0,3.0,0.0,22.0,1.0,0.0,7.2500,0.0,0.0,1.0
1,2.0,1.0,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,0.0
2,3.0,1.0,3.0,1.0,26.0,0.0,0.0,7.9250,0.0,0.0,1.0
3,4.0,1.0,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,1.0
4,5.0,0.0,3.0,0.0,35.0,0.0,0.0,8.0500,0.0,0.0,1.0


In [143]:
data_train = data[:700]
data_test = data[700:]


In [144]:
# 訓練データを少数クラスのデータと多数クラスに分割
data_train_1 = data_train[data_train['Survived'] == 1]
data_train_0 = data_train[data_train['Survived'] == 0]

In [145]:
data_train_1_feature = data_train_1.loc[:, 'Pclass':'Embarked_S'].values
data_train_1_target = data_train_1.loc[:, 'Survived'].values
data_train_0_feature = data_train_0.loc[:, 'Pclass':'Embarked_S'].values
data_train_0_target = data_train_0.loc[:, 'Survived'].values

data_test_feature = data_test.loc[:, 'Pclass':'Embarked_S'].values
data_test_target = data_test.loc[:, 'Survived'].values

<h2>ニューラルネットワークモデル定義</h2>
<h4>GridSearchCVとかPipelineが便利なので、sklearnのBaseEstimatorを継承しました</h4>

In [115]:
from sklearn.base import BaseEstimator
from sklearn.base import BaseEstimator, ClassifierMixin

OUTPUT_SIZE=2

class skNN(BaseEstimator,nn.Module,ClassifierMixin):
    def __init__(self, input_layer=20,hidden_layer=20,lr=0.01):
        self.input_layer = input_layer
        self.hidden_layer = hidden_layer
        self.lr = lr
        
        super(skNN, self).__init__()
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer)
        self.fc2 = nn.Linear(self.hidden_layer,self.hidden_layer)
        self.fc3 = nn.Linear(self.hidden_layer,OUTPUT_SIZE)
        self.outact = nn.Softmax()
        
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(),lr=self.lr)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.dropout(x)
        x = self.fc3(x)
        return self.outact(x)
        
    def fit(self, X, y=None, **params):
        self.input_layer = len(X[0])
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer)
        
        tensorX = torch.from_numpy(X).float()
        tensorY = torch.from_numpy(y).long()
        train = TensorDataset(tensorX,tensorY)
        train_loader = DataLoader(train, batch_size=32, shuffle=True)

        for epoch in range(int(len(X)/32)):
            total_loss = 0    
            for train_x, train_y in train_loader:
                train_x, train_y = Variable(train_x), Variable(train_y)
                self.optimizer.zero_grad()
                output = self(train_x)
                loss = self.criterion(output, train_y)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.data[0]
            
            if (epoch+1) % 50 == 0:
                print(epoch+1, total_loss)
        return self
    
    def predict(self, X):
        tensorX = torch.from_numpy(X).float()
        test_X = Variable(tensorX)
        return (self(test_X).data[:,1].numpy() > 0.5).astype(int)
    
    def predict_proba(self, X):
        tensorX = torch.from_numpy(X).float()
        test_X = Variable(tensorX)
        return self(test_X).data[:,1].numpy()
    
    def score(self,X,y):
        tensorX = torch.from_numpy(X).float()
        test_X = Variable(tensorX)
        return sum( (self(test_X).data[:,1].numpy() > 0.5).astype(int) == y )/ len(y)
    
    def get_params(self,deep=True):
        return {'hidden_layer': self.hidden_layer, 'lr': self.lr}
    
    def set_params(self,**params):
        for parameter, value in params.items():
            setattr(self,parameter, value)
        self.fc1 = nn.Linear(self.input_layer,self.hidden_layer)
        self.fc2 = nn.Linear(self.hidden_layer,self.hidden_layer)
        self.fc3 = nn.Linear(self.hidden_layer,OUTPUT_SIZE)
        self.optimizer = optim.Adam(self.parameters(),lr=self.lr)
        return self
        

<h2>学習実行</h2>
<h4>折角なので色んな種類の識別器を使ってみます。それぞれでグリッドサーチし、最適なパラメータを探索しています</h4>

In [129]:
# 識別器の定義
from sklearn.linear_model import LogisticRegression
from sklearn import svm
clfs = {#"GB1":GradientBoostingClassifier(),
        #"GB2":GradientBoostingClassifier(),
        #"GB3":GradientBoostingClassifier(),
        #"GB4":GradientBoostingClassifier(),
        #"GB5":GradientBoostingClassifier(),
        #"GB6":GradientBoostingClassifier(),
        #"GB7":GradientBoostingClassifier(),
        #"GB8":GradientBoostingClassifier(),
        #"GB9":GradientBoostingClassifier(),
        #"GB10":GradientBoostingClassifier()
        "NN1":skNN(),
        "NN2":skNN(),
        "NN3":skNN(),
        #"NN4":skNN(),
        #"NN5":skNN()
       }


In [130]:
# 各Estimatorのパラメータ探索用dict
gb_parameters = {
    "select__threshold" : ["mean", "median"],
    "clf__learning_rate": [0.01],
    "clf__max_depth":[5,7],
    "clf__subsample":[0.5, 1.0],
    "clf__n_estimators":[150,100,200]
}
rf_parameters = {
    "clf__n_estimators" : [100],
    "clf__max_features" : [1, 'auto', None],
    "clf__max_depth" : [1, 5, 10, None],
    "clf__min_samples_leaf": [1, 2, 4,]
}
lr_parameters = {
    "clf__C": [0.01,0.1,1]
}
svc_parameters = {
    "clf__C": [1,10,100],
    "clf__gamma": [0.01,0.1,1]
}
nn_parameters = {
    "select__threshold" : ["mean", "median"],
    "clf__hidden_layer": [30,40,50,60],
    "clf__lr": [0.01,0.05],
}

In [147]:
data_train_1.shape

(271, 11)

In [153]:
data_train_0.head(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1.0,0.0,3.0,0.0,22.0,1.0,0.0,7.2500,0.0,0.0,1.0
4,5.0,0.0,3.0,0.0,35.0,0.0,0.0,8.0500,0.0,0.0,1.0
5,6.0,0.0,3.0,0.0,28.0,0.0,0.0,8.4583,0.0,1.0,0.0
6,7.0,0.0,1.0,0.0,54.0,0.0,0.0,51.8625,0.0,0.0,1.0
7,8.0,0.0,3.0,0.0,2.0,3.0,1.0,21.0750,0.0,0.0,1.0


In [156]:
data_train_0.values[:,1:]

array([[ 0.,  3.,  0., ...,  0.,  0.,  1.],
       [ 0.,  3.,  0., ...,  0.,  0.,  1.],
       [ 0.,  3.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  3.,  0., ...,  0.,  0.,  1.],
       [ 0.,  1.,  0., ...,  1.,  0.,  0.],
       [ 0.,  3.,  0., ...,  0.,  0.,  1.]])

In [157]:
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from imblearn.pipeline import Pipeline 
i=0

# 識別器格納用リスト
gb_clfs = []

# 各識別器に対して学習実行
for clf in clfs.items():
    i += 1
    print(i) 
    
    # 多数データを少数データの数だけUnderSamplingし、少数データとマージ
    train_data_ = np.vstack((data_train_0.sample(n=int(len(data_train_1))),data_train_1.sample(n=int(len(data_train_1)))))

    # マージ後の訓練データをシャッフル
    np.random.shuffle(train_data_)
    # 訓練データを説明変数と目的変数に分割
    train_data_feature = train_data_[:,2:].astype(float)
    train_data_target = train_data_[:,1].astype(float)
    
    # 説明変数(特徴量)抽出モデルと標準化モデルを定義
    select = SelectFromModel(RandomForestClassifier(n_estimators=100, n_jobs=-1))
    pca = PCA()
    sm = SMOTE()
    scaler = StandardScaler()
    
    # 一連の処理(Pipeline)を定義
    # 特徴量抽出→標準化→識別器訓練
    estimator = [
        ('select',select),
        #('pca',pca),
        #('smote' ,sm),
        ('scaler',scaler),
        ('clf',clf[1])
    ]

    pipe = Pipeline(estimator)
    
    # グリッドサーチ用モデルの定義
    # 識別器の種類によってパラメータが異なるので条件分岐
    if clf[0] == 'SVC':
        gb_clf = GridSearchCV(pipe, svc_parameters, scoring='f1', cv=3, n_jobs=-2)
    elif clf[0] == 'RF':
        gb_clf = GridSearchCV(pipe, rf_parameters, scoring='f1', cv=3, n_jobs=-2)
    elif clf[0] =='NN':
        gb_clf = GridSearchCV(pipe, nn_parameters, scoring='f1', cv=3, n_jobs=-1)
    elif clf[0].find('LR') > -1:
        gb_clf = GridSearchCV(pipe, lr_parameters, scoring='f1', cv=3, n_jobs=-1)
    else:
        gb_clf = GridSearchCV(pipe, gb_parameters, scoring='f1', cv=3, n_jobs=-1)

    # グリッドサーチ実行
    gb_clf.fit(data_train_feature,data_train_target)
    
    # 最適なパラメータで訓練された識別器をリストに追加
    gb_clfs.append(gb_clf)

1


/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension ch

/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension ch

/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension ch

/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension ch

...

3


/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be a

/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension ch

/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarn

/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension ch

<h2>評価</h2>

In [158]:
for clf in gb_clfs:
    print(clf.best_params_)

{'clf__learning_rate': 0.01, 'clf__max_depth': 7, 'clf__n_estimators': 100, 'clf__subsample': 1.0, 'select__threshold': 'median'}
{'clf__learning_rate': 0.01, 'clf__max_depth': 5, 'clf__n_estimators': 200, 'clf__subsample': 0.5, 'select__threshold': 'median'}
{'clf__learning_rate': 0.01, 'clf__max_depth': 7, 'clf__n_estimators': 200, 'clf__subsample': 0.5, 'select__threshold': 'median'}


In [159]:
# 投票箱作成
predict_vote=np.zeros(int(len(data_test_feature)))
predict_proba_vote=np.zeros(int(len(data_test_feature)))

# 各識別器の予測結果を投票箱に入れる
for gb_clf in gb_clfs:
    predict_vote += gb_clf.predict(data_test_feature)
#    predict_proba_vote += gb_clf.predict_proba(data_test_feature)[:,1]
    
# 投票結果は多数決    
predict_vote = (predict_vote/len(gb_clfs) > 0.5).astype(int)
#predict_proba_vote = (predict_proba_vote/len(gb_clfs))
predict_vote

/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


array([1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0])

In [160]:
sum(data_test_target == predict_vote)/len(data_test_target)

0.83246073298429324

In [164]:
train_data_.shape

(542, 11)

In [161]:
# F値確認
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

print(f1_score(predict_vote,data_test_target))
confusion_matrix(predict_vote,data_test_target)

0.753846153846


array([[110,  22],
       [ 10,  49]])

In [170]:
data2 = pd.read_csv("./data/test.csv").drop('Ticket',axis=1).drop('Cabin',axis=1).drop('Name',axis=1)
data2["Sex"][data2["Sex"] == "male"] = 0
data2["Sex"][data2["Sex"] == "female"] = 1
PassengerId = data2['PassengerId']

data2["Age"] = data2["Age"].fillna(data2["Age"].median())
data2["Fare"] = data2["Fare"].fillna(data2["Fare"].median())
data2["Embarked"] = data2["Embarked"].fillna("S")

dummy = pd.get_dummies(data2[dummy_columns])
data2 = pd.concat([data2, dummy], axis=1).drop('Embarked',axis=1).astype(float)

data2_feature = data2.loc[:, 'Pclass':'Embarked_S'].values

/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [171]:
# 投票箱作成
predict_vote_test=np.zeros(int(len(data2_feature)))
predict_proba_vote_test=np.zeros(int(len(data2_feature)))

# 各識別器の予測結果を投票箱に入れる
for gb_clf in gb_clfs:
    predict_vote_test += gb_clf.predict(data2_feature)
#    predict_proba_vote_test += gb_clf.predict_proba(data2_feature)[:,1]
    
# 投票結果は多数決    
predict_vote_test = (predict_vote_test/len(gb_clfs) > 0.5).astype(int)
#predict_proba_vote_test = (predict_proba_vote_test/len(gb_clfs))
predict_vote_test

/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [172]:
predict_vote_test.shape

(418,)

In [173]:
BaggingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predict_vote_test })

In [174]:
BaggingSubmission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [175]:
BaggingSubmission.to_csv("BaggingSubmission.csv", index=False)